# Model Training for Flood Detection

This notebook is designed to train a convolutional neural network (CNN) for flood detection using SAR images. The training process includes data loading, preprocessing, augmentation, and model evaluation.

## idées sur entrainement à explorer


Calibration radiométrique (DN → σ⁰) – Si ce n’est pas déjà fait, il faut convertir les valeurs brutes du TIFF (DN en amplitude) en valeurs physiquement calibrées. Typiquement, on applique la formule de calibration avec la LUT fournie pour obtenir l’intensité σ⁰ de chaque pixel (souvent exprimée en décibels). Cette étape met toutes les images sur une échelle comparable. Dans certains cas, on peut se contenter de travailler en « β⁰ non corrigé de l’angle », mais pour la classification d’inondation, le σ⁰ en dB est préférable afin que par exemple un sol sec à 33° et le même sol à 45° d’incidence aient des valeurs comparables après normalisation angulaire.


Correction du bruit thermique – Les produits Sentinel‑1 contiennent des profils de bruit de fond (thermal noise) estimés, notamment pour les extrémités de swath où le signal utile est faible. Il est recommandé de soustraire ce bruit de fond (fourni dans les annotations sous forme de LUT de bruit en range/azimut) avant ou après la calibration​
CERWEB.IFREMER.FR. Cela remet à zéro le niveau de référence des pixels qui n’ont pratiquement aucun signal rétrodiffusé, évitant de confondre du bruit avec un faible retour réel (important pour ne pas classer l’océan ou des zones réellement sans signal comme de l’eau peu profonde). Concrètement, on calcule σ⁰_calibré = (DN² / A²) – σ⁰_bruit, en veillant à ne pas obtenir de valeurs négatives (les pixels où le bruit dépasse le signal utile peuvent être mis à NaN ou à 0).


Filtrage du speckle – Comme discuté en section 5, il est souvent bénéfique d’appliquer un filtre de speckle modéré sur l’image intensité (exemple : un filtre de Lee 3×3 ou 5×5). Pour une IA, cela peut aider à se concentrer sur les structures larges (plaques d’eau, berges, etc.) plutôt que sur le grain. Toutefois, un excès de lissage risque de supprimer de petites mares ou des détails fins utiles à la décision. Une bonne pratique est d’adapter la force du filtre à l’échelle du phénomène d’intérêt : pour des inondations s’étalant sur des dizaines de pixels, un filtre 3×3 réduit suffisamment le bruit sans effacer les zones étroites. À l’inverse, si l’architecture de l’IA intègre déjà des mécanismes pour traiter le bruit (par ex. un CNN avec de la pooling pourrait lisser implicitement), on peut choisir un filtrage plus léger. L’objectif est que le niveau de speckle résiduel ne génère pas de confusion dans la phase d’apprentissage (par exemple, éviter qu’un pixel d’eau brillamment bruité soit appris comme “non-inondé”).


Masquage des zones non pertinentes – Cette étape dépend du contexte, mais pour de la détection d’inondation on souhaite souvent masquer certains pixels avant la classification automatique :
Bordures et valeurs invalides : les zones en dehors de l’emprise utile (bord noir de l’image TIFF s’il y en a), ou les pixels où la calibration n’est pas définie (quelques lignes en début/fin d’image parfois). Ces pixels doivent être masqués (valeur nodata) pour ne pas induire l’IA en erreur.
Océan et eaux permanentes : si l’objectif est de détecter les inondations terrestres, il est judicieux de masquer l’océan et éventuellement les grands plans d’eau permanents (lacs, cours d’eau habituels). En effet, ceux-ci apparaissent également sombres comme des inondations, mais ne sont pas du « terrain inondé nouvellement ». On peut utiliser une masque d’eau statique (ex. à partir de la couche ESA WorldCover ou d’une classification antérieure Sentinel‑2) pour exclure ces zones de l’analyse ou au moins informer le modèle (certains modèles concatènent ce masque en entrée supplémentaire).
Zones d’ombre radar ou à incidence extrême : En terrain montagneux, les images SAR comportent des ombres (zones que le radar n’a pas pu éclairer, derrière une colline par exemple) et des zones de layover (chevauchement où l’écho de la pente arrive en même temps que celui de la vallée). Ces pixels peuvent apparaître sombres (ombre) ou brillants (fausses superpositions) de manière trompeuse. Il peut être utile de les repérer via un modèle numérique de terrain et de les masquer ou au moins de ne pas les considérer comme candidats à l’inondation (une ombre radar n’est pas de l’eau, même si c’est sombre). De même, les tout premiers pixels proches (incidence faible) ou les pixels en bout de swath (incidence très élevée) peuvent avoir des caractéristiques différentes (bruit plus fort, projection plus incertaine) – on peut choisir d’ignorer par prudence, par ex., les 1–2% des pixels les plus en bord d’image.


Changement d’échelle des valeurs (normalisation) – Pour faciliter l’apprentissage de l’IA, on applique en général une normalisation des canaux en entrée. Si l’on travaille en dB, on peut par exemple recentrer les valeurs autour de –20 dB et les diviser par 10, ou appliquer un min-max pour les ramener dans [0,1] ou [–1,1]. L’important est que les distributions de valeurs en entrée du réseau soient à peu près homogènes entre les différentes images d’entraînement, afin que le modèle ne soit pas perturbé par des décalages (par exemple une image globale plus brillante suite à une pluie uniforme sur la zone – ce cas peut être partiellement compensé par une normalisation locale ou globale). La normalisation peut aussi impliquer de passer en échelle logarithmique si ce n’est pas déjà fait (beaucoup de modèles préfèrent travailler avec les valeurs en dB plutôt que linéaires, car la distribution en dB est plus gaussienne).


Extraction de caractéristiques additionnelles – Bien que le réseau de deep learning puisse apprendre directement sur les canaux VV et VH bruts (après calibrations et filtrages ci-dessus), on enrichit parfois l’entrée avec des features dérivées pour guider l’IA. Dans le cas de deux polarisation, une caractéristique courante est le ratio polarimétrique VH/VV (ou VH–VV en dB) par pixel. Cette ratio met en évidence les différences de comportement polarimétrique indépendamment de l’intensité absolue. Par exemple, une surface inondée aura à la fois VV et VH faibles, donc un ratio VH/VV ~ 1 (0 dB) en tendance, tandis qu’une végétation aura VH plus faible que VV (ratio < 1). D’autres caractéristiques possibles : des textures (calculées via matrices de co-occurrence GLCM sur l’image dB) pour détecter le voisinage homogène de l’eau vs l’hétérogénéité du sol végétalisé – toutefois, dans les approches CNN modernes, c’est généralement le réseau lui-même qui déduira ces textures via ses filtres convolutifs. Si l’on utilise un modèle de classification non convolutif, ajouter des indicateurs de texture ou des statistiques locales peut être bénéfique. On pourrait également injecter des données auxiliaires : par ex. l’altitude du terrain (MNT) pour aider à savoir si une zone sombre dans une vallée très basse est plus susceptible d’être de l’eau qu’une zone sombre en flanc de montagne (ombre). En résumé, toute information qui peut aider à différencier l’eau du non-eau de manière cohérente peut être ajoutée comme couche supplémentaire au modèle.


Division en ensembles et augmentation – Pour l’apprentissage supervisé, il faut préparer les patchs d’entraînement ou mosaïques, en s’assurant d’un bon équilibre de classes (inondé vs non) et en appliquant éventuellement de l’augmentation de données (rotations, ajouts de bruit, etc.) pour rendre le modèle robuste. Ce point dépasse le simple prétraitement d’une image individuelle, mais fait partie intégrante du pipeline de préparation des données avant IA.

In [2]:
import os
import sys
import importlib

def setup_path():
    """
    Configure les chemins d'accès pour permettre l'import des modules du projet.
    Ajoute le répertoire racine du projet au sys.path pour que les imports 
    de modules comme 'src' fonctionnent correctement, peu importe d'où le script est exécuté.
    
    Retourne:
        bool: True si la configuration a réussi, False sinon
    """
    # Dans un notebook, __file__ n'est pas défini, on utilise getcwd() à la place
    try:
        current_dir = os.path.dirname(os.path.abspath(__file__))
    except NameError:
        current_dir = os.getcwd()
        
    print(f"Répertoire courant: {current_dir}")
    
    # Essayer de trouver le répertoire racine du projet (jusqu'à 3 niveaux au-dessus)
    root_dir = current_dir
    max_levels = 3
    
    for _ in range(max_levels):
        # Vérifier si nous sommes à la racine du projet
        if os.path.exists(os.path.join(root_dir, 'src')):
            # Nous avons trouvé le répertoire racine
            if root_dir not in sys.path:
                sys.path.append(root_dir)
                print(f"Ajout de {root_dir} au sys.path")
            
            # Vérifier si l'import fonctionne maintenant
            try:
                # Tenter d'importer un module pour vérifier
                importlib.import_module('src')
                print("✅ Configuration des chemins réussie.")
                return True
            except ImportError as e:
                print(f"❌ Échec de l'import après ajout au sys.path: {e}")
        
        # Remonter d'un niveau
        parent_dir = os.path.dirname(root_dir)
        if parent_dir == root_dir:  # Nous sommes déjà à la racine du système de fichiers
            break
        root_dir = parent_dir
    
    # Si nous arrivons ici, nous n'avons pas trouvé le répertoire racine
    # Essayer une dernière approche en ajoutant le parent direct
    last_resort = os.path.abspath(os.path.join(current_dir, '..'))
    if last_resort not in sys.path:
        sys.path.append(last_resort)
        print(f"Tentative de dernier recours: ajout de {last_resort} au sys.path")
    
    # Vérifier une dernière fois
    try:
        importlib.import_module('src')
        print("✅ Configuration des chemins réussie (dernier recours).")
        return True
    except ImportError as e:
        print(f"❌ Échec de la configuration des chemins: {e}")
        print("Structure de projet attendue:")
        print("project_root/")
        print("├── src/")
        print("│   ├── data/")
        print("│   ├── models/")
        print("│   └── ...")
        print("└── notebooks/")
        return False

In [3]:
setup_path()

Répertoire courant: c:\Users\mokht\Desktop\PDS\flood_dataset\flood-detection-cnn\notebooks
Ajout de c:\Users\mokht\Desktop\PDS\flood_dataset\flood-detection-cnn au sys.path
✅ Configuration des chemins réussie.


True

In [4]:
import torch
import os
from torch.utils.data import random_split
from src.data.dataloader import SARDataset
from src.models.cnn import FloodDetectionCNN, train_flood_detection_model
import matplotlib.pyplot as plt

In [5]:

def main():
    print("Démarrage de l'entraînement du modèle de détection d'inondations...")
    
    # Définir les chemins
    dataset_path = "sar_dataset.pkl"
    checkpoints_dir = "checkpoints"
    model_save_path = "flood_detection_model.pth"
    
    # Créer le répertoire de checkpoints s'il n'existe pas
    os.makedirs(checkpoints_dir, exist_ok=True)
    
    # 1. Utiliser la méthode create_dataloader pour charger les données
    # Si le dataset existe déjà
    if os.path.exists(dataset_path):
        print(f"Chargement du dataset depuis {dataset_path}...")
        
        # Option 1: Utiliser directement les dataloaders pour train/val
        train_loader, train_dataset = SARDataset.create_dataloader(
            use_saved_dataset=True,
            saved_dataset_path=dataset_path,
            batch_size=32,
            num_workers=4,
            shuffle=True
        )
        
        val_loader, val_dataset = SARDataset.create_dataloader(
            use_saved_dataset=True,
            saved_dataset_path=dataset_path,
            batch_size=32,
            num_workers=4,
            shuffle=False
        )
        
        test_loader, test_dataset = SARDataset.create_dataloader(
            use_saved_dataset=True,
            saved_dataset_path=dataset_path,
            batch_size=32,
            num_workers=4,
            shuffle=False
        )
        
        # Option 2: Alternative - Charger le dataset complet puis le diviser
        # Ce qui permet une meilleure séparation train/val/test
        print("Division du dataset en ensembles d'entraînement/validation/test...")
        dataset = SARDataset.load(dataset_path)
        print(f"Dataset chargé avec {len(dataset)} échantillons")
        
        # 2. Diviser le dataset en train/val/test
        train_size = int(0.7 * len(dataset))
        val_size = int(0.15 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        
        # Utiliser une graine fixe pour la reproductibilité
        generator = torch.Generator().manual_seed(42)
        train_dataset, val_dataset, test_dataset = random_split(
            dataset, [train_size, val_size, test_size], generator=generator
        )
        
        print(f"Répartition: {train_size} entraînement, {val_size} validation, {test_size} test")
        
        # Créer les dataloaders à partir des sous-ensembles
        train_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True
        )
        val_loader = torch.utils.data.DataLoader(
            val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True
        )
        test_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True
        )
    
    else:
        # Si le dataset n'existe pas encore, il faut le créer à partir du CSV
        print(f"Création du dataset à partir du CSV...")
        train_loader, train_dataset = SARDataset.create_dataloader(
            batch_size=32, 
            num_workers=4,
            shuffle=True,
            save_after_loading=True,
            saved_dataset_path=dataset_path
        )
        
        # Dans ce cas, nous utilisons uniquement train_loader pour entraîner un modèle initial
        # Idéalement, nous voudrions quand même diviser les données
        print("ATTENTION: Vous devriez exécuter ce script à nouveau après création du dataset")
        print("pour pouvoir le diviser correctement en ensembles train/val/test.")
        
        # Pour l'exemple, nous utilisons le même loader pour val et test (ce n'est pas optimal)
        val_loader = train_loader
        test_loader = train_loader
    
    # 3. Configurer et entraîner le modèle CNN
    print("\nConfiguration et entraînement du modèle CNN...")
    
    # Paramètres du modèle
    model_params = {
        'input_channels': 2,  # VH et VV
        'num_classes': 2,     # Inondé ou non inondé
        'dropout_rate': 0.5   # Pour réduire le surapprentissage
    }
    
    # Paramètres d'entraînement
    training_params = {
        'num_epochs': 30,
        'learning_rate': 0.001,
        'weight_decay': 1e-5,
        'checkpoint_dir': checkpoints_dir,
        'early_stopping_patience': 5
    }
    
    # Entraîner le modèle avec la fonction utilitaire
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Utilisation de l'appareil: {device}")
    
    model, history, metrics = train_flood_detection_model(
        train_loader, val_loader, test_loader,
        model_params=model_params,
        training_params=training_params
    )
    
    # 4. Sauvegarder le modèle final
    model.save(model_save_path)
    print(f"Modèle sauvegardé à {model_save_path}")
    
    # 5. Afficher un résumé des résultats
    print("\nRésumé des performances du modèle:")
    print(f"Exactitude: {metrics['accuracy']:.4f}")
    print(f"Précision: {metrics['precision']:.4f}")
    print(f"Rappel: {metrics['recall']:.4f}")
    print(f"Score F1: {metrics['f1']:.4f}")
    
    # Visualiser les résultats
    model.visualize_training_history(history)
    model.visualize_results(metrics)
    
    return model, history, metrics

main()

Démarrage de l'entraînement du modèle de détection d'inondations...
Chargement du dataset depuis sar_dataset.pkl...
Chargement du dataset depuis sar_dataset.pkl...
Dataset chargé: 3331 échantillons
Chargement du dataset depuis sar_dataset.pkl...
Dataset chargé: 3331 échantillons
Chargement du dataset depuis sar_dataset.pkl...
Dataset chargé: 3331 échantillons
Division du dataset en ensembles d'entraînement/validation/test...
Chargement du dataset depuis sar_dataset.pkl...
Dataset chargé: 3331 échantillons
Dataset chargé avec 3331 échantillons
Répartition: 2331 entraînement, 499 validation, 501 test

Configuration et entraînement du modèle CNN...
Utilisation de l'appareil: cpu
Modèle créé avec 33648610 paramètres
Entraînement sur cpu pendant 30 époques


Époque 1/30 [Train]:  11%|█         | 8/73 [00:46<06:13,  5.75s/it, loss=nan, acc=0]


KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = Dataset(split='train')
val_dataset = Dataset(split='val')

# Data augmentation
train_dataset = augment_data(train_dataset)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=training_config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=training_config['batch_size'], shuffle=False)

In [ ]:
# Initialize model
model = FloodDetectionCNN(**model_config)
model.to(device)

# Initialize trainer
trainer = Trainer(model, train_loader, val_loader, training_config)

# Train the model
trainer.train()

In [ ]:
# Evaluate the model
val_metrics = calculate_metrics(trainer.model, val_loader, device)
print(val_metrics)

## Conclusion

In this notebook, we have successfully trained a CNN for flood detection using SAR images. The model's performance has been evaluated on the validation set, and metrics have been reported.